Both!

In [6]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
import cProfile
import re


%matplotlib inline

the spin rate of the earth changes too
start off assuming steady state, that the moon's angular velocities in the inertial frame and the earth's angular velocities in the inertial frame are the same, so in a little time dt, the bulge and moon maintain the same seperation.

Then use change in phi* and theta* for the moon to calculate its new angular velocity about the earth.
Then use conservation of angular momentum to calculate the earth's new angular velocity. 
delta is then equal to delta + or - the little angle rotated by the moon minus the little angle rotated by the earth

delta = delta0 + (omegaMoon * t - omegaEarth * t)

In the even more general case, model the evolution tidal deformation as a satellite starts off far away and comes closer. rederive the formulas, i think. 

*Implement by 1/17/2026

**Check equations with Dr. Johnston 

In [ ]:
#20x faster than numpy..?
def cross(a, b):
    return np.array([a[1]*b[2] - a[2]*b[1], -a[0]*b[2] + a[2]*b[0], a[0]*b[1] - a[1]*b[0]])

def findInertiaTensor(a, b, c, m):
    #inertia tensor over a,b,c
    tensor = np.zeros((3))
    #closed form expression
    #diagonals of the tensor
    tensor[0] = m*(b**2 + c**2)/5
    tensor[1] = m*(a**2 + c**2)/5
    tensor[2] = m*(b**2 + a**2)/5
    return tensor

def findScaledInertiaTensor(a, b, c, m):
    tensor = np.zeros((3))
    #by symmetry, off-diagonals vanich
    # Tij = ri * rj * ||r||^2
    tensor[0] = 3*m*( (a**4)/35 + ((a*b)**2 + (a*c)**2)/105 )
    tensor[1] = 3*m*( (b**4)/35 + ((b*a)**2 + (b*c)**2)/105 )
    tensor[2] = 3*m*( (c**4)/35 + ((c*a)**2 + (c*b)**2)/105 )
    return tensor

def globalToLocalRotate(r, localQuaternion):
    #rotate a globally defined vector into a locally defined coordinate system
    #returns rotated vector
    q0 = localQuaternion[0]
    qv = localQuaternion[1:4]
    t = 2.0 * cross(qv, r)
    return r - q0 * t + cross(qv, t)

def localToGlobalRotate(r, localQuaternion):
    #rotate a locally defined vector into a globally defined coordinate system
    #returns rotated vector
    q0 = localQuaternion[0]
    qv = localQuaternion[1:4]
    t = 2.0 * cross(qv, r)
    return r + q0 * t + cross(qv, t)

def findTorqueVector(Rcm, ThetaCM, RcmMagnitude, theta, DELTA, m, GAMMA_R_CUBED, InertiaTensor, MU): 
    P2 = 3 * (np.cos(theta - DELTA))**2 - 1
    kappa = np.sin(2*theta - 2 * DELTA)

    IRcm = np.multiply(InertiaTensor, Rcm)
    #SIRcm = np.multiply(ScaledInertiaTensor, Rcm) 

    m5 = 1 / RcmMagnitude**5
    m10 = 5 / (RcmMagnitude**10)

    # USE a+b+c = sum of diagonals over 2
    #Dividing by different magnitudes because we are using unit vectors and the derived equations use location vectors. 
    torque1 = (3 * MU * m5) * cross(Rcm, IRcm)
    torque2 = (m10) * cross(Rcm, IRcm) #second order
    #torque3 = (m7) * cross(Rcm, SIRcm) #third order

    torque4 = (m10) * cross(ThetaCM, IRcm) #first order
    torque5 = (-m10 * m * (InertiaTensor[0]+InertiaTensor[1]+InertiaTensor[2])) * cross(ThetaCM, Rcm) #second order
    #torque6 = (m7) * cross(ThetaCM, SIRcm) #third order

    return torque1 + GAMMA_R_CUBED * P2 * (torque2 ) + GAMMA_R_CUBED * kappa * (torque4 + torque5 )

def findTheta_hat_cm(R_hat_cm, q_p):
    #Takes Rhat
    #Rotates into primary
    #Convert to theta
    #rotate bacl
    primary_frame_R = globalToLocalRotate(R_hat_cm, q_p)
    sqrt = np.sqrt(primary_frame_R[0]**2 + primary_frame_R[1]**2)
    theta_Hat_local = np.array([primary_frame_R[2] * primary_frame_R[0] / sqrt,
                                primary_frame_R[2] * primary_frame_R[1] / sqrt,
                                sqrt])
    return localToGlobalRotate(theta_Hat_local, q_p)
    



In secondary, 
$$
sin(\theta) = \frac{\sqrt{x^2 + y^2}}{\sqrt{x^2 +y^2 + z^2}}, 
cos(\theta) = \frac{z}{\sqrt{x^2 +y^2 + z^2}},
sin(\phi) = \frac{y}{\sqrt{x^2 +y^2}},
cos(\phi) = \frac{x}{\sqrt{x^2 +y^2}}
$$ 

$$
\begin{pmatrix}
x \\
y \\
z
\end{pmatrix}

=
\begin{pmatrix}
sin(\theta)cos(\phi) \\
sin(\theta)sin(\phi) \\
cos(\theta)
\end{pmatrix}
$$
Rotate
$$
\begin{pmatrix}
cos(\theta)cos(\phi) \\
cos(\theta)sin(\phi) \\
sin(theta)
\end{pmatrix}
=
\begin{pmatrix}
\frac{zx}{\sqrt{(x^2+y^2+z^2)(x^2+y^2)}} \\
\frac{zy}{\sqrt{(x^2+y^2+z^2)(x^2+y^2)}} \\
\frac{\sqrt{x^2 + y^2}}{\sqrt{x^2 +y^2 + z^2}}
\end{pmatrix}
$$


Unit vectors $\implies$

$$
\begin{pmatrix}
\frac{zx}{\sqrt{(x^2+y^2+z^2)(x^2+y^2)}} \\
\frac{zy}{\sqrt{(x^2+y^2+z^2)(x^2+y^2)}} \\
\frac{\sqrt{x^2 + y^2}}{\sqrt{x^2 +y^2 + z^2}}
\end{pmatrix}
=
\begin{pmatrix}
\frac{zx}{\sqrt{(x^2+y^2)}} \\
\frac{zy}{\sqrt{(x^2+y^2)}} \\
\sqrt{x^2 + y^2}
\end{pmatrix}
$$

In [ ]:
def ODE(t, state, G, M, m, GAMMA_R_CUBED, InertiaTensorSecondary, ScaledInertiaTensorSecondary, InertiaTensorPrimary):

    dydt = np.empty(20)
    #-----------Secondary Body-------------#
    #Used multiple times, denoted here to calculate it only once
    
    #Points to secondary relative to primary in global coordinates
    cosTheta = np.cos(state[1])
    sinTheta = np.sin(state[1])
    cosPhi = np.cos(state[2])
    sinPhi = np.sin(state[2])

    q_s = state[9:13] / np.sqrt( state[9]**2 + state[10]**2 + state[11]**2 + state[12]**2 )
    q_p = state[16:20] / np.sqrt( state[16]**2 + state[17]**2 + state[18]**2 + state[19]**2 )

    #Locates COM of primary 
    R_hat_cm = np.array([sinTheta*cosPhi, sinTheta*sinPhi, cosTheta])

    #Corresponding ThetaCM
    Theta_hat_CM = findTheta_hat_cm(R_hat_cm, q_p)
    
    #Primary's orientation relative to the inertial frame, alligned so (0,0,1) locates the tidal bulge, and the remaining vectors locate the tidal minima
    #Find position of secondary in this frame to calculate DELTA
    DELTA = np.arccos(centerOfMassUnitSecondaryLocal[2])
    kappa = np.sin(2*(state[2] - DELTA))
    
    #The torque in the body frame of the secondary
    torque = findTorqueVector(centerOfMassUnitSecondaryLocal,
                              globalToLocalRotate(centerOfMassThetaUnitGlobal, q_p),
                              state[0], state[1], DELTA, m, GAMMA_R_CUBED, 
                              InertiaTensorSecondary, ScaledInertiaTensorSecondary, MU=G*M)
    
    #Magnitude of the torque on the secondary in the inertial frame about the primary
    kinematicTorque = (GAMMA_R_CUBED * kappa / state[0]**8)

    #Kinematic evolution of the secondary in spherical coordinates
    #Secondary's mass not used because it cancels in each of these instances
    dydt[0] = state[1]
    dydt[1] = state[0] * state[3]**2 - (G*M) / state[0]**2 - (GAMMA_R_CUBED / state[0]**7) * (3 * np.cos(state[2] - DELTA)**2 - 1)
    dydt[2] = state[3]
    dydt[3] = -2 * state[1] * state[3] / state[0] - kinematicTorque
    dydt[4] = state[5]
    dydt[5] = -2 * state[1] * state[5] / state[0]

    #Evolution of dynamic angular velocity of secondary in the body frame
    dydt[6] = ((InertiaTensorSecondary[1]-InertiaTensorSecondary[2]) * state[7] * state[8] + torque[0]) / InertiaTensorSecondary[0]
    dydt[7] = ((InertiaTensorSecondary[2]-InertiaTensorSecondary[0]) * state[8] * state[6] + torque[1]) / InertiaTensorSecondary[1]
    dydt[8] = ((InertiaTensorSecondary[0]-InertiaTensorSecondary[1]) * state[6] * state[7] + torque[2]) / InertiaTensorSecondary[2]

    #Evolution of the secondary's quaternion
    dydt[9]  = -0.5 * (q_s[1]*state[6] + q_s[2]*state[7] + q_s[3]*state[8]) 
    dydt[10] =  0.5 * (q_s[0]*state[6] - q_s[3]*state[7] + q_s[2]*state[8])
    dydt[11] =  0.5 * (q_s[3]*state[6] + q_s[0]*state[7] - q_s[1]*state[8])
    dydt[12] =  0.5 * (q_s[2]*state[6] + q_s[1]*state[7] - q_s[0]*state[8])


    #-----------Primary Body-------------#
    #Torque exerted on primary is the opposite of the net torques exerted on the secondary by CoAM
    #Net torque exerted on secondary is the kinematic torque about the primary's COM (points in decreasing theta direction) and dynamic torque about secondary's COM
    #Frame is alligned so Z-axis points directly at the secondary, and X and Y-axis point along min(Iy, Iz) and max(Iy, Iz) respectively
    #As long as delta is small, we can assume the inertia tensor to almost be diagonal. 
    #When specifying initial conditions, ensure the omega's and inertia tensor allign.
    net_torque = globalToLocalRotate(localToGlobalRotate(torque, q_s), q_p) #rotate secondary body -> global -> primary body
    net_torque+= globalToLocalRotate(state[0]**2 * m * kinematicTorque * centerOfMassThetaUnitGlobal, q_p)

    #Evolution of primary's angular velocity
    dydt[13] = ((InertiaTensorPrimary[1]-InertiaTensorPrimary[2]) * state[14] * state[15] - net_torque[0]) / InertiaTensorPrimary[0]
    dydt[14] = ((InertiaTensorPrimary[2]-InertiaTensorPrimary[0]) * state[15] * state[13] - net_torque[1]) / InertiaTensorPrimary[1]
    dydt[15] = ((InertiaTensorPrimary[0]-InertiaTensorPrimary[1]) * state[13] * state[14] - net_torque[2]) / InertiaTensorPrimary[2]

    #Evolution of primary's orientation
    dydt[16] = -0.5 * (q_p[1]*state[13] + q_p[2]*state[14] + q_p[3]*state[15])
    dydt[17] =  0.5 * (q_p[0]*state[13] - q_p[3]*state[14] + q_p[2]*state[15])
    dydt[18] =  0.5 * (q_p[3]*state[13] + q_p[0]*state[14] - q_p[1]*state[15])
    dydt[19] =  0.5 * (q_p[2]*state[13] + q_p[1]*state[14] - q_p[0]*state[15])

    #derivatives of state vector
    return dydt

In [33]:
numYears = 1
T = numYears * 31556952 
sampleNumber = 100
t_eval = range(0, T, int(T/sampleNumber))

#True Constants
G = 6.67 * 10**-11
MASSPRIMARY = 5.972 * 10**24
MASSSECONDARY = 7.3 * 10**22
SHEAR_MODULUS = 150 * 10**9
MEAN_PRIMARY_RADIUS = 6371 * 1000
#a < b < c
a_s = 1736 * 1000
b_s = 1738 * 1000
c_s = 1738 * 1000

a_p = 6357 * 1000
b_p = 6378 * 1000
c_p = 6378 * 1000

#Calculated Constants
mean_density = MASSPRIMARY / (4 * 3.1415 * a_p * b_p * c_p / 3)
surface_gravity = G * MASSPRIMARY / MEAN_PRIMARY_RADIUS**2
effective_rigidity = (19 * SHEAR_MODULUS) / (2 * mean_density * surface_gravity * MEAN_PRIMARY_RADIUS)
GAMMA_R_CUBED = (9 * G * MASSSECONDARY * MEAN_PRIMARY_RADIUS**5) / (4 * (1 + effective_rigidity))

InertiaTensorSecondary = findInertiaTensor(a_s, b_s, c_s, MASSSECONDARY)
ScaledInertiaTensorSecondary = findScaledInertiaTensor(a_s, b_s, c_s, MASSSECONDARY)

InertiaTensorPrimary = findInertiaTensor(a_p, b_p, c_p, MASSPRIMARY)

#-----------Initial State Specifications-------------#

#Secondary
r =  1000000
rd = 0
theta = np.pi
thetad = 100
phi = np.pi
phid = 50
omega_1 = 0
omega_2 = 0
omega_3 = 0
q_0 = 1
q_1 = 0

q_2 = 0
q_3 = 0

#Primary
omega_1_primary = 0
omega_2_primary = 0
omega_3_primary = 0
q_0_primary = 1
q_1_primary = 0
q_2_primary = 0
q_3_primary =  0

state_naught = [r, rd, theta, thetad, phi, phid, omega_1, omega_2, omega_3, q_0, q_1, q_2, q_3, omega_1_primary, omega_2_primary, omega_3_primary, q_0_primary, q_1_primary, q_2_primary, q_3_primary]

timePoints = np.linspace(0, T, sampleNumber) 

sol = integrate.solve_ivp(rtol= .01, fun=ODE, t_span=(0, T), y0=state_naught, args=(G, MASSPRIMARY, MASSSECONDARY, GAMMA_R_CUBED, InertiaTensorSecondary, ScaledInertiaTensorSecondary, InertiaTensorPrimary))

#8.3
#
print(len(sol.t))

C:\Users\aarus\AppData\Local\Temp\ipykernel_58172\4027389031.py:24: RuntimeWarning: invalid value encountered in arccos
  DELTA = np.arccos(centerOfMassUnitSecondaryLocal[2])


820


Potential is given by 
$$\chi(r, \theta, \phi) = -\frac{GM_p}{r} - \frac{3Gm_sR_p^5}{4(1+\~{\mu})r^3a^3}(3cos^2(\theta-\delta)-1)$$
$M_p$ = mass primary <br>
$m_s$ = mass secondary<br>
$R_p$ = mean radius of primary<br>
$r$ = radial coordinate of point<br>
$\theta$ = angle from +z. We take z+ to always point along the line connecting the primary's COM to the secondary's COM <br>
$\phi$ = azimuthal angle from x+ <br>
$a$ = distance between centers of mass<br>
$\delta$ = phase lag<br>
$\~{\mu}$ = effective ridgidity = $\frac{19\mu}{ 2\rho g R_p}$<br>
$\mu$ = shear modulus<br>
$\rho$ = mean density<br>
$g$ = surface gravity<br>

The force is given by
$$-m_s\nabla\chi(r, \theta) = -\left(\frac{GM_pm_s}{r^2} + \frac{9Gm_s^2R_p^5}{4(1+\~{\mu})r^4a^3}(3cos^2(\theta-\delta)-1)\right)\hat{r} - \frac{1}{r}\left(\frac{9Gm_s^2R_p^5}{4(1+\~{\mu})r^3a^3}sin(2(\theta-\delta))\right)\hat\theta$$
Here,
$$\hat{r} = \begin{pmatrix} sin(\theta)cos(\phi) \\ sin(\theta)sin(\phi) \\ cos(\theta) \end{pmatrix} \text{ increasing } r$$
$$\hat{\theta} = \begin{pmatrix} cos(\theta)cos(\phi) \\ cos(\theta)sin(\phi) \\ sin(\theta) \end{pmatrix} \text{ decreasing }\theta $$
$$\hat{\phi} = \begin{pmatrix} -sin(\phi) \\ cos(\phi) \\ 0 \end{pmatrix} \text{ increasing } \phi$$

Henceforth I denote <br>
$\gamma = \frac{9Gm_sR_p^5}{4(1+\~{\mu})a^3}$
<br>
$P_2 = 3cos^2(\theta-\delta)-1$
<br> 
$\kappa = \frac{\partial{P_2}}{\partial{\theta}} = sin(2(\theta-\delta))$

so that

$$F(r, \theta, \phi) = -m_s\nabla\chi(r, \theta, \phi) = -\left(\frac{GM_pm_s}{r^2} + \frac{\gamma P_2 m_s}{r^4}\right)\hat{r} - \left(\frac{\gamma \kappa m_s}{r^4}\right)\hat\theta$$

The torque $\tau$ can be calculated using
$$\tau = \int{r' \times dF}$$
Where $r'$ locates a particle force $dF$ acts on relative to the body's center of mass. 

For our first term, 
$$\tau_1 = \int{r' \times - \frac{GM_p}{r^2}dm}$$
Schaube and Junkins showed that
$$\tau_1 = \frac{3GM_p}{R_c^5}\vec{R_c}\times [I]\vec{R_c}$$
Where $R_c$ is the distance between the centers of masses and $\vec{R_c}$ is the vector that locates the secondary's COM in the orbit frame centered on the primary. 


I will now approximate the torque for the other radial term and the tangential term.

The body frame is centered on the secondary's COM and has three orthogonal axis pointing along the principal moments of inertia $I_x, I_y, Iz$ where I assume $I_x < I_y < I_z$, and that the secondary may be represented as an ellipsoid. The orbit frame has it's origin on the COM of the primary. The z-axis points in the direction of the secondary's COM. Hence, we have a rotating frame. 

$$\tau_2 = \int{\vec{r'} \times dF} = -\gamma P_2 \int{\frac{\vec{r'} \times \hat{r}}{||R||^4} dm}$$
and
$$\tau_3 = \int{\vec{r'} \times dF} = -\gamma \kappa \int{\frac{\vec{r'} \times \hat{\theta}}{||R||^4} dm}$$

Such that $$\tau_{net} = \tau_1 + \tau_2 + \tau_3$$

Notationally, $r'$ locates mass element $dm$ relative to the Body's COM. $\hat{r}$ is the unit vector pointing the the secondary's COM in the orbit frame. $\vec{R}$ is the position of the secondary's COM in the orbit frame. $R_c$ and $\vec{R_c}$ are defined previously.

I use $\hat{r} = \frac{\vec{R}}{||R||}$ and $\vec{R} = \vec{R_c} + \vec{r'}$ so that $\hat{r} = \frac{\vec{R_c} + \vec{r'}}{||R||}$

$$\tau_2 = -\gamma P_2 \int{\frac{\vec{r'} \times (\vec{R_c} + \vec{r'})}{||R||^5} dm}$$
$$= -\gamma P_2 \int{\frac{\vec{r'} \times \vec{R_c}}{||R||^5} dm}$$
$$= \gamma P_2 \vec{R_c} \times \int{\frac{\vec{r'}}{||R||^5} dm}$$

I used that $\vec{r'} \times \vec{r'} = 0$ and the fact that $\vec{R_c}$ is constant over each $dm$. 

$$||R||^5 = ||\vec{R_c} + \vec{r'}||^5 = (R_c^2 + 2\vec{R_c} \cdot \vec{r'} + r'^2)^{\frac{-5}{2}}$$
Keep in mind the vector signage. 

$$= \frac{1}{R_c^5} \left(1 + \frac{2\vec{R_c} \cdot \vec{r'}}{R_c^2} + \frac{r'^2}{R_c^2}\right)^{\frac{-5}{2}}$$

Binomially expanding to the third degree, 

$$= 1 - \frac{5}{2}\left(\frac{2\vec{R_c} \cdot \vec{r'}}{R_c^2} + \frac{r'^2}{R_c^2}\right) + \frac{35}{8} \left(\frac{4(\vec{R_c} \cdot \vec{r'})^2}{R_c^4} + \frac{4(\vec{R_c} \cdot \vec{r'})(r'^2)}{R_c^4} + \frac{r'^4}{R_c^4}\right)$$

$$= \frac{\gamma P_2}{R_c^5} \vec{R_c} \times \int{\frac{\vec{r'}}{||R||^5} dm} = \frac{\gamma P_2}{R_c^5} \vec{R_c} \times \int{\vec{r'} \left(
    1 
    - \frac{5}{2}\left(\frac{2\vec{R_c} \cdot \vec{r'}}{R_c^2} + 
    \frac{r'^2}{R_c^2}\right) + 

    \frac{35}{8} \left(\frac{4(\vec{R_c} \cdot \vec{r'})^2}{R_c^4} + 
    \frac{4(\vec{R_c} \cdot \vec{r'})(r'^2)}{R_c^4} + 
    \frac{r'^4}{R_c^4}\right) dm \right)}$$

<br>

$$
=
\frac{\gamma P_2}{R_c^5}  \vec{R_c} \times \int{\vec{r'}dm} 
$$

$$
-
\frac{5\gamma P_2}{R_c^7}   \vec{R_c} \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})dm} -
\frac{5\gamma P_2}{2R_c^7}  \vec{R_c} \times \int{\vec{r'} r'^2dm}
$$

$$
+
\frac{35\gamma P_2}{2R_c^9}  \vec{R_c} \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})^2dm}+
\frac{35\gamma P_2}{2R_c^9}  \vec{R_c} \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})(r'^2)dm}+
\frac{35\gamma P_2}{8R_c^9}  \vec{R_c} \times \int{\vec{r'} r'^4dm}+
$$

<br>

By the definition of COM. The integrand is odd in $r'$ for each of the following:
$$\frac{\gamma P_2}{R_c^5}  \vec{R_c} \times \int{\vec{r'}dm}$$
$$\frac{5\gamma P_2}{2R_c^7}  \vec{R_c} \times \int{\vec{r'} r'^2dm}$$
$$\frac{35\gamma P_2}{2R_c^9}  \vec{R_c} \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})^2dm}$$
$$\frac{35\gamma P_2}{8R_c^9}  \vec{R_c} \times \int{\vec{r'} r'^4dm} $$

So each of these terms evaluates to $0$ by symmetry. 

We are left with 
$$\tau_2 = -\frac{5\gamma P_2}{R_c^7}   \vec{R_c} \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})dm} + \frac{35\gamma P_2}{2R_c^9}  \vec{R_c} \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})(r'^2)dm}$$

Again, Schaube and Junkins have shown that
$$\vec{R_c} \times\int{\vec{r'}(\vec{R_c} \cdot \vec{r'})dm} = \vec{R_c} \times\int{\vec{r'} \times (\vec{r'} \times \vec{R_c})dm} + \vec{R_c} \times\int{\vec{R_c}r'^2dm} = \vec{R_c} \times-[I]\vec{R_c} + 0$$
where $[I]$ is the standard inertia tensor. 

Now we are left with the following integral:
$$\int{\vec{r'} (\vec{R_c} \cdot \vec{r'})(r'^2)dm}$$

I define this as the scaled inertia tensor, $[T]$, where $$T_{ij} = \int{r'_ir'_j(r'^2)dm}$$
so that $$\int{\vec{r'} (\vec{R_c} \cdot \vec{r'})(r'^2)dm} = [T]\vec{R_c}$$

In the body frame, $$T_{ij} = 0 \text{ for } i\not=j$$

Thus, we are left with $$T_{ii} = \int{(r'_i)^2(r'^2)dm}$$

Evaluating $T_{xx}$, 

$$T_{xx} = \rho\int{x^2(x^2 + y^2 + z^2)dV}$$
over the region $\frac{x^2}{a^2} + \frac{y^2}{b^2} + \frac{z^2}{c^2} = 1$
where $a > b > c$. 

Use the change of variables 
$$x = au$$
$$y = bv$$
$$z = cw$$
$$J = abc$$

Thus, 
$$T_{xx} = \rho abc\int_{u^2 + v^2 + w^2 = 1}{a^2u^2(a^2u^2 + b^2v^2 + c^2w^2)dudvdw}$$
$$ = abc\int_{u^2 + v^2 + w^2 = 1}{a^4u^4 + a^2b^2u^2v^2 + a^2c^2u^2w^2dudvdw}$$

Next, we turn to spherical coordinates:
$$u = psin(\theta)cos(\phi)$$
$$v = psin(\theta)sin(\phi)$$
$$w = pcos(\theta)$$
$$J = p^2sin(\theta)$$

So, 

$$T_{xx} = \rho abc\int_{0}^{2\pi} \int_{0}^{\pi} \int_{0}^{1}{
   [a^4p^6sin^4(\theta)cos^4(\phi)
 + a^2b^2p^6sin^4(\theta)cos^2(\phi)sin^2(\phi)
 + a^2c^2p^6sin^2(\theta)cos(\phi)^2cos^2(\theta)]sin(\theta)

 drd\theta d\phi}
$$

$$
=
\rho a^5bc\int_{0}^{2\pi} \int_{0}^{\pi} \int_{0}^{1}{    p^6sin^5(\theta)cos^4(\phi) drd\theta d\phi}
$$
$$
+\rho a^3b^3c\int_{0}^{2\pi} \int_{0}^{\pi} \int_{0}^{1}{  p^6sin^5(\theta)cos^2(\phi)sin^2(\phi) drd\theta d\phi}
$$
$$
+\rho a^3bc^3\int_{0}^{2\pi} \int_{0}^{\pi} \int_{0}^{1}{      p^6sin^3(\theta)cos^2(\theta)cos(\phi)^2 drd\theta d\phi}
$$ 

<br>

$$=
\rho a^5bc
\int_{0}^{2\pi}{cos^4(\phi) d\phi}
\int_{0}^{\pi}{sin^5(\theta)d\theta} 
\int_{0}^{1}{p^6dr}
$$
$$+
\rho a^3b^3c
\int_{0}^{2\pi}{cos^2(\phi)sin^2(\phi) d\phi}
\int_{0}^{\pi}{sin^5(\theta)d\theta} 
\int_{0}^{1}{p^6dr}
$$
$$+
\rho a^3bc^3
\int_{0}^{2\pi}{cos^2(\phi) d\phi}
\int_{0}^{\pi}{sin^3(\theta)cos^2(\theta)d\theta} 
\int_{0}^{1}{p^6dr}
$$

$$=
\rho a^5bc*
\frac{3\pi}{4}*
\frac{16}{15}*
\frac{1}{7}
$$
$$+
\rho a^3b^3c*
\frac{\pi}{4}*
\frac{16}{15}*
\frac{1}{7}
$$
$$+
\rho a^3bc^3*
\pi*
\frac{4}{15}*
\frac{1}{7}
$$

=

$$=
\rho a^5bc
\frac{12\pi}{105}
+
\rho a^3b^3c
\frac{4\pi}{105}
+
\rho a^3bc^3
\frac{4\pi}{105}
$$

For an ellipse, $\frac{3}{4}M = \pi \rho abc$
$$T_{xx} =  Ma^2(\frac{3a^2}{35}+\frac{b^2}{35}+\frac{c^2}{35})$$
$$T_{yy} =  Mb^2(\frac{a^2}{35}+\frac{3b^2}{35}+\frac{c^2}{35})$$
$$T_{zz} =  Mc^2(\frac{a^2}{35}+\frac{b^2}{35}+\frac{3c^2}{35})$$


Substituting in everything,

$$\tau_2 = \frac{5\gamma P_2}{R_c^7}   \vec{R_c} \times [I]\vec{R_c} + \frac{35\gamma P_2}{2R_c^9}  \vec{R_c} \times [T]\vec{R_c}$$

$\gamma$ has units $\frac{N\cdot m^4}{kg}$, $P_2$ is unitless, $[I]$ has units $kg\cdot m^2$,  $[T]$ has units $kg \cdot m^4$, and $\tau_2$ has units $N \cdot m$. 
As a sanity check, we find that
$$ N \cdot m = \frac{N\cdot m^4}{kg} \cdot \frac{1}{m^7} \cdot m \cdot kg\cdot m^2 \cdot m + \frac{N\cdot m^4}{kg} \cdot \frac{1}{m^9} \cdot m \cdot kg\cdot m^4 \cdot m$$
$$ N \cdot m = \frac{N \cdot m^8 \cdot kg}{kg \cdot m^7} + \frac{N \cdot m^9 \cdot kg}{kg \cdot m^8}$$
$$N \cdot m = N \cdot m $$

Finally, 

$$\tau_3 = \int{\vec{r'} \times dF} = -\gamma \kappa \int{\frac{\vec{r'} \times \hat{\theta}}{||R||^4} dm}$$

I choose the polar coordinate to range from $0$ to $2\pi$. 
Previously, we had $\vec{r'}$, $\vec{R_c}$, and $\vec{R}$. Now, it helps to introduce $\vec{\theta'}$, $\vec{\theta _c}$, and $\vec{\theta}$.
We use a great deal of coordinates, so here is how one converts from a vector $\vec{R}_{general}$ to its corresponding $\vec{\theta}_{general}$. First, the position vector is rotated into the primary frame. Next, it is rotated $-\frac{\pi}{2}$ radians in the $\theta$ direction. This is why we chose to define $\hat{\theta}$ opposite to what one would expect.

Essentially, the corresponding $\theta$-vector for any given position vector is simply given by that position vector rotated towards the line connecting the primary and secondary's COM. This is most easily represented in the primary frame. 

All of the following calculations will be taken in the primary frame. 

$$\tau_3 = -\gamma \kappa \int{\frac{\vec{r'} \times \hat{\theta}}{||R||^4} dm} $$
$$= -\gamma \kappa \int{\frac{\vec{r'} \times \vec{\theta}}{||R||^5} dm}$$
$$= -\gamma \kappa \int{\frac{\vec{r'} \times (\vec{\theta}_{cm} + \vec{\theta}')}{||R||^5} dm}$$

Where $$\vec{\theta} = \vec{\theta}_{cm} + \vec{\theta}'$$.
We can only choose two of these vectors to be rotations of their corresponding position vectors. In general, the third will not be representable as a rotation of it's corresponding position vector. It is natural to define
$$\vec{\theta} \cdot \vec{R} = 0$$
and
$$\vec{\theta}_{cm} \cdot \vec{R}_{cm} = 0$$
Then, in general, 
$$\vec{\theta}' \cdot \vec{r'} \not= 0$$

However, since we have
$$||\vec{\theta}'|| << ||\vec{\theta}_{cm}|| \text{ and } ||\vec{\theta}'|| << ||\vec{\theta}||$$
and 
$$||\vec{r}'|| << ||\vec{R}_{cm}|| \text{ and } ||\vec{r}'|| << ||\vec{R}||$$


We can assume $\vec{\theta}'$ is nearly perpendicular to $\vec{r}'$:
The error of this assumption is proportional to the azimuthal angle $\phi$ between the $\vec{R}$ and $\vec{R}_{cm}$. When $\phi = 0$, then $\vec{\theta}' \cdot \vec{r'} = 0$. If the magnitude of $\phi$ is small (as is the case for small magnitude $\vec{r}'$ and $\vec{\theta}'$) then  $\vec{\theta}' \cdot \vec{r'} \approx 0$. So, $\hat{\theta}' \times \hat{r'} = \hat{\phi}$


With this information, 

$$\tau_3 = -\gamma \kappa \int{\frac{\vec{r'} \times (\vec{\theta}_{cm} + \vec{\theta}')}{||R||^5} dm}$$
$$ = -\gamma \kappa \int{\frac{\vec{r'} \times \vec{\theta}_c}{||R||^5} dm}$$

Skipping some steps, 

$$
=
\frac{\gamma \kappa}{R_c^5}  \vec{\theta}_c \times \int{\vec{r'}dm} 
$$

$$
-
\frac{5\gamma \kappa}{R_c^7}   \vec{\theta}_c \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})dm} -
\frac{5\gamma \kappa}{2R_c^7}  \vec{\theta}_c \times \int{\vec{r'} r'^2dm}
$$

$$
+
\frac{35\gamma \kappa}{2R_c^9}  \vec{\theta}_c \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})^2dm}+
\frac{35\gamma \kappa}{2R_c^9}  \vec{\theta}_c \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})(r'^2)dm}+
\frac{35\gamma \kappa}{8R_c^9}  \vec{\theta}_c \times \int{\vec{r'} r'^4dm}
$$

Similarly, 

$$
\frac{\gamma \kappa}{R_c^5}  \vec{\theta}_c \times \int{\vec{r'}dm} 
$$
$$
\frac{5\gamma \kappa}{2R_c^7}  \vec{\theta}_c \times \int{\vec{r'} r'^2dm}
$$
$$
\frac{35\gamma \kappa}{2R_c^9}  \vec{\theta}_c \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})^2dm}
$$
$$
\frac{35\gamma \kappa}{8R_c^9}  \vec{\theta}_c \times \int{\vec{r'} r'^4dm}
$$
each drop out by parity arguements, and we're left with: 

$$
\tau_3 = -\frac{5\gamma \kappa}{R_c^7}   \vec{\theta}_c \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})dm} + \frac{35\gamma \kappa}{2R_c^9}  \vec{\theta}_c \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})(r'^2)dm} $$

$$ = -\frac{5\gamma \kappa}{R_c^7}   \vec{\theta}_c \times \int{\vec{r'}\times (\vec{r'} \times \vec{R}_c ) - \frac{5\gamma \kappa}{R_c^7}}   \vec{\theta}_c \times  \int{\vec{R_c}r'^2dm} + \frac{35\gamma \kappa}{2R_c^9}  \vec{\theta}_c \times \int{\vec{r'} (\vec{R_c} \cdot \vec{r'})(r'^2)dm}  $$

$$= \frac{5\gamma \kappa}{R_c^7}   \vec{\theta}_c \times [I]\vec{R}_c - \frac{5\gamma \kappa}{R_c^7}   \vec{\theta}_c \times \vec{R_c}\int{r'^2dm} + \frac{35\gamma \kappa}{2R_c^9}  \vec{\theta}_c \times [T]\vec{R_c}$$

$$ = \frac{5\gamma \kappa}{R_c^7}   \vec{\theta}_c \times [I]\vec{R}_c - \frac{m_s(a^2 + b^2 + c^2)\gamma \kappa}{R_c^7}   \vec{\theta}_c \times \vec{R_c} + \frac{35\gamma \kappa}{2R_c^9}  \vec{\theta}_c \times [T]\vec{R_c}$$

Remember, this gives you a massive vector. In the program, I will find $\vec{R_c}$ in $x, y, z$ components, rotate it into the primary frame using a quaternion, rotate it into $\vec{\theta}_c$ and then rotate it from the orbital to the secondary frame, as the Inertia matrix takes its simplest form in this frame. 

Finally, 

$$\tau_{net} = \tau_1 + \tau_2 + \tau_3$$
$$ = \frac{3GM_p}{R_c^5}\vec{R_c}\times [I]\vec{R_c} +\frac{5\gamma P_2}{R_c^7}   \vec{R_c} \times [I]\vec{R_c} + \frac{35\gamma P_2}{2R_c^9}  \vec{R_c} \times [T]\vec{R_c} +\frac{5\gamma \kappa}{R_c^7}   \vec{\theta}_c \times [I]\vec{R}_c - \frac{m_s(a^2 + b^2 + c^2)\gamma \kappa}{R_c^7}   \vec{\theta}_c \times \vec{R_c} + \frac{35\gamma \kappa}{2R_c^9}  \vec{\theta}_c \times [T]\vec{R_c}$$

to the third order, or

$$ = \frac{3GM_p}{R_c^5}\vec{R_c}\times [I]\vec{R_c}+\frac{5\gamma P_2}{R_c^7}   \vec{R_c} \times [I]\vec{R_c} +  \frac{5\gamma \kappa}{R_c^7}\vec{\theta}_c \times [I]\vec{R}_c - \frac{m_s(a^2 + b^2 + c^2)\gamma \kappa}{R_c^7}   \vec{\theta}_c \times \vec{R_c}$$

to the second order, which is the minimum required for both components of the tidal potential to exhibit a torque on the secondary. 
Expanding $\gamma$, $P_2$, and $\kappa$, 
$$ = \frac{3GM_p}{R_c^5}\vec{R_c}\times [I]\vec{R_c} +\frac{45Gm_sR_p^5}{4(1+\~{\mu})R_c^{10}}(3cos^2(\theta-\delta)-1)   \vec{R_c} \times [I]\vec{R_c} +  \frac{45Gm_sR_p^5}{4(1+\~{\mu})R_c^{10}} sin(2(\theta-\delta))\vec{\theta}_c \times [I]\vec{R}_c - \frac{9(a^2 + b^2 + c^2)Gm_s^2R_p^5}{4(1+\~{\mu})R_c^{10}} sin(2(\theta-\delta))   \vec{\theta}_c \times \vec{R_c}$$

Redefining
$\gamma = \frac{9Gm_sR_p^5}{4(1+\~{\mu})}$
<br>
$P_2 = (3cos^2(\theta-\delta)-1)$
<br> 
$\kappa = \frac{\partial{P_2}}{\partial{\theta}} = sin(2(\theta-\delta))$
yields

$$= \frac{3GM_p}{R_c^5}\vec{R_c}\times [I]\vec{R_c} +\frac{5\gamma P_2}{R_c^{10}}   \vec{R_c} \times [I]\vec{R_c} +  \frac{5\gamma \kappa}{R_c^{10}}\vec{\theta}_c \times [I]\vec{R}_c - \frac{m_s(a^2 + b^2 + c^2)\gamma \kappa}{R_c^{10}}   \vec{\theta}_c \times \vec{R_c}$$


In [ ]:
def findTorqueVector(Rcm, ThetaCM, RcmMagnitude, theta, DELTA, m, GAMMA_R_CUBED, InertiaTensor, MU): 
    P2 = 3 * (np.cos(theta - DELTA))**2 - 1
    kappa = np.sin(2*theta - 2 * DELTA)

    IRcm = np.multiply(InertiaTensor, Rcm)
    #SIRcm = np.multiply(ScaledInertiaTensor, Rcm) 

    m5 = 1 / RcmMagnitude**5
    m10 = 5 / (RcmMagnitude**10)

    # USE a+b+c = sum of diagonals over 2
    #Dividing by different magnitudes because we are using unit vectors and the derived equations use location vectors. 
    torque1 = (3 * MU * m5) * cross(Rcm, IRcm)
    torque2 = (m10) * cross(Rcm, IRcm) #second order
    #torque3 = (m7) * cross(Rcm, SIRcm) #third order

    torque4 = (m10) * cross(ThetaCM, IRcm) #first order
    torque5 = (-m10 * m * (InertiaTensor[0]+InertiaTensor[1]+InertiaTensor[2])) * cross(ThetaCM, Rcm) #second order
    #torque6 = (m7) * cross(ThetaCM, SIRcm) #third order

    return torque1 + GAMMA_R_CUBED * P2 * (torque2 ) + GAMMA_R_CUBED * kappa * (torque4 + torque5 )

Tomorrow: rewrite code.



take the vector finding COM in global
take vector locating delta
to rotate vectors using quaternion, find the cross product and the angle between the two vectors. then rotate about that rotation axis by the specified amount. 

Rehash equations given that theta = 0 corresponds to vector locating COM (additional symmetries? what does theta CM look like?)


 the documentation of the equations is complete, a section explaining code might be worth. A section deriving lagrangian equations of motion is imperative. finally, input the data for the earth-moon system, and graph some stuff! 
